# Classification Alzheimer - Guide Complet
## Dataset: Images IRM pour la détection des stades d'Alzheimer

### Classes:
- **NonDemented**: Pas de démence
- **VeryMildDemented**: Démence très légère
- **MildDemented**: Démence légère
- **ModerateDemented**: Démence modérée

---

## Étape 1: Importation des bibliothèques

In [ ]:
# Bibliothèques de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# TensorFlow et Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, GlobalAveragePooling2D, Input
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, 
    TensorBoard, CSVLogger
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3

# Métriques et évaluation
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, precision_score, recall_score, f1_score
)

# Configuration
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Seed pour la reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

## Étape 2: Configuration des chemins et paramètres

In [ ]:
# Chemins des données
BASE_DIR = Path('AlzheimerDataset')
TRAIN_DIR = BASE_DIR / 'train'
TEST_DIR = BASE_DIR / 'test'

# Chemins de sauvegarde
MODEL_DIR = Path('AlzheimerModel')
MODELS_DIR = MODEL_DIR / 'models'
GRAPHS_DIR = MODEL_DIR / 'graphs'
LOGS_DIR = MODEL_DIR / 'logs'

# Créer les dossiers s'ils n'existent pas
for directory in [MODELS_DIR, GRAPHS_DIR, LOGS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Paramètres du modèle
IMG_SIZE = (176, 176)  # Taille des images
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001

# Classes
CLASSES = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
NUM_CLASSES = len(CLASSES)

print(f"Train Directory: {TRAIN_DIR}")
print(f"Test Directory: {TEST_DIR}")
print(f"Number of Classes: {NUM_CLASSES}")
print(f"Classes: {CLASSES}")

## Étape 3: Exploration des données

In [ ]:
# Fonction pour compter les images par classe
def count_images(directory):
    """Compte le nombre d'images dans chaque classe"""
    data = {'Class': [], 'Count': []}
    
    for class_name in CLASSES:
        class_path = directory / class_name
        if class_path.exists():
            count = len(list(class_path.glob('*.jpg'))) + len(list(class_path.glob('*.png')))
            data['Class'].append(class_name)
            data['Count'].append(count)
            print(f"{class_name}: {count} images")
    
    return pd.DataFrame(data)

# Compter les images
print("\n=== TRAIN SET ===")
train_df = count_images(TRAIN_DIR)

print("\n=== TEST SET ===")
test_df = count_images(TEST_DIR)

print(f"\nTotal Train Images: {train_df['Count'].sum()}")
print(f"Total Test Images: {test_df['Count'].sum()}")

## Étape 4: Visualisation de la distribution des données

In [ ]:
# Visualiser la distribution des classes
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Train set
sns.barplot(data=train_df, x='Class', y='Count', ax=axes[0], palette='viridis')
axes[0].set_title('Distribution des classes - Train Set', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Classe', fontsize=12)
axes[0].set_ylabel('Nombre d\'images', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)

# Test set
sns.barplot(data=test_df, x='Class', y='Count', ax=axes[1], palette='magma')
axes[1].set_title('Distribution des classes - Test Set', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Classe', fontsize=12)
axes[1].set_ylabel('Nombre d\'images', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(GRAPHS_DIR / 'class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## Étape 5: Visualisation d'exemples d'images

In [ ]:
# Afficher des exemples d'images pour chaque classe
fig, axes = plt.subplots(4, 4, figsize=(15, 15))

for i, class_name in enumerate(CLASSES):
    class_path = TRAIN_DIR / class_name
    images = list(class_path.glob('*.jpg')) + list(class_path.glob('*.png'))
    
    # Afficher 4 images par classe
    for j in range(4):
        if j < len(images):
            img = cv2.imread(str(images[j]))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[i, j].imshow(img)
            axes[i, j].set_title(f'{class_name}', fontsize=10)
        axes[i, j].axis('off')

plt.suptitle('Exemples d\'images IRM par classe', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(GRAPHS_DIR / 'sample_images.png', dpi=300, bbox_inches='tight')
plt.show()

## Étape 6: Préparation des données avec augmentation

In [ ]:
# Générateur d'augmentation pour les données d'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,                # Normalisation
    rotation_range=20,             # Rotation aléatoire
    width_shift_range=0.2,         # Décalage horizontal
    height_shift_range=0.2,        # Décalage vertical
    horizontal_flip=True,          # Retournement horizontal
    zoom_range=0.2,                # Zoom aléatoire
    shear_range=0.2,               # Cisaillement
    fill_mode='nearest',           # Remplissage des pixels
    validation_split=0.2           # 20% pour validation
)

# Générateur pour les données de test (seulement normalisation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Créer les générateurs
print("Création des générateurs de données...\n")

# Train generator
train_generator = train_datagen.flow_from_directory(
    str(TRAIN_DIR),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Validation generator
validation_generator = train_datagen.flow_from_directory(
    str(TRAIN_DIR),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    str(TEST_DIR),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\nClasses: {train_generator.class_indices}")

## Étape 7: Construction du modèle CNN

In [ ]:
def create_cnn_model(input_shape=(*IMG_SIZE, 3), num_classes=NUM_CLASSES):
    """
    Crée un modèle CNN personnalisé pour la classification Alzheimer
    """
    model = Sequential([
        # Premier bloc convolutionnel
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Deuxième bloc convolutionnel
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Troisième bloc convolutionnel
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Quatrième bloc convolutionnel
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Couches denses
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        # Couche de sortie
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Créer le modèle
model = create_cnn_model()

# Afficher l'architecture
model.summary()

## Étape 8: Compilation du modèle

In [ ]:
# Compiler le modèle
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', 
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

print("Modèle compilé avec succès!")

## Étape 9: Configuration des callbacks

In [ ]:
# Définir les callbacks
callbacks = [
    # Sauvegarde du meilleur modèle
    ModelCheckpoint(
        filepath=str(MODELS_DIR / 'best_model.h5'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    # Arrêt anticipé si pas d'amélioration
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Réduction du learning rate si plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    # TensorBoard pour visualisation
    TensorBoard(
        log_dir=str(LOGS_DIR),
        histogram_freq=1
    ),
    
    # Sauvegarde de l'historique en CSV
    CSVLogger(
        filename=str(LOGS_DIR / 'training_log.csv'),
        separator=',',
        append=False
    )
]

print("Callbacks configurés:")
for callback in callbacks:
    print(f"  - {callback.__class__.__name__}")

## Étape 10: Entraînement du modèle

In [ ]:
# Calculer les steps
STEPS_PER_EPOCH = train_generator.samples // BATCH_SIZE
VALIDATION_STEPS = validation_generator.samples // BATCH_SIZE

print(f"Steps per epoch: {STEPS_PER_EPOCH}")
print(f"Validation steps: {VALIDATION_STEPS}")
print(f"\nDébut de l'entraînement...\n")

# Entraîner le modèle
history = model.fit(
    train_generator,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS,
    callbacks=callbacks,
    verbose=1
)

print("\nEntraînement terminé!")

## Étape 11: Visualisation de l'historique d'entraînement

In [ ]:
# Fonction pour tracer l'historique
def plot_training_history(history):
    """
    Trace les courbes d'apprentissage
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 0].set_title('Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 1].set_title('Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
    axes[1, 0].set_title('Precision', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
    axes[1, 1].set_title('Recall', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(GRAPHS_DIR / 'training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

# Tracer l'historique
plot_training_history(history)

## Étape 12: Évaluation sur le test set

In [ ]:
# Évaluer sur le test set
print("Évaluation sur le test set...\n")

test_loss, test_acc, test_precision, test_recall = model.evaluate(
    test_generator,
    verbose=1
)

# Calculer le F1-score
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print("\n" + "="*50)
print("RÉSULTATS SUR LE TEST SET")
print("="*50)
print(f"Test Accuracy:  {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss:      {test_loss:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1-Score:  {test_f1:.4f}")
print("="*50)

## Étape 13: Matrice de confusion

In [ ]:
# Prédictions sur le test set
print("Génération des prédictions...")
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)

# Visualiser la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=CLASSES,
    yticklabels=CLASSES,
    cbar_kws={'label': 'Nombre de prédictions'}
)
plt.title('Matrice de Confusion', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Vraie classe', fontsize=12)
plt.xlabel('Classe prédite', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(GRAPHS_DIR / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Matrice de confusion normalisée
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm_normalized, 
    annot=True, 
    fmt='.2%', 
    cmap='YlOrRd',
    xticklabels=CLASSES,
    yticklabels=CLASSES,
    cbar_kws={'label': 'Pourcentage'}
)
plt.title('Matrice de Confusion Normalisée', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Vraie classe', fontsize=12)
plt.xlabel('Classe prédite', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(GRAPHS_DIR / 'confusion_matrix_normalized.png', dpi=300, bbox_inches='tight')
plt.show()

## Étape 14: Rapport de classification détaillé

In [ ]:
# Rapport de classification
print("\n" + "="*70)
print("RAPPORT DE CLASSIFICATION DÉTAILLÉ")
print("="*70)
print(classification_report(y_true, y_pred, target_names=CLASSES))
print("="*70)

## Étape 15: Sauvegarde du modèle final

In [ ]:
# Sauvegarder le modèle complet
model_path = MODELS_DIR / 'alzheimer_model_final.h5'
model.save(str(model_path))
print(f"✓ Modèle complet sauvegardé: {model_path}")

# Sauvegarder les poids uniquement
weights_path = MODELS_DIR / 'alzheimer_weights.h5'
model.save_weights(str(weights_path))
print(f"✓ Poids sauvegardés: {weights_path}")

# Sauvegarder l'architecture en JSON
architecture_path = MODELS_DIR / 'model_architecture.json'
with open(architecture_path, 'w') as f:
    f.write(model.to_json())
print(f"✓ Architecture sauvegardée: {architecture_path}")

# Sauvegarder l'historique d'entraînement
history_df = pd.DataFrame(history.history)
history_csv_path = LOGS_DIR / 'training_history.csv'
history_df.to_csv(history_csv_path, index=False)
print(f"✓ Historique sauvegardé: {history_csv_path}")

## Étape 16: Fonction de prédiction pour nouvelles images

In [ ]:
def predict_alzheimer_stage(image_path, model, img_size=IMG_SIZE):
    """
    Prédit le stade d'Alzheimer pour une nouvelle image IRM
    
    Args:
        image_path: Chemin vers l'image
        model: Modèle entraîné
        img_size: Taille de l'image
    
    Returns:
        dict: Prédictions avec probabilités pour chaque classe
    """
    # Charger et prétraiter l'image
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    # Prédiction
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = CLASSES[predicted_class_idx]
    confidence = predictions[0][predicted_class_idx] * 100
    
    # Créer un dictionnaire avec toutes les probabilités
    probabilities = {CLASSES[i]: float(predictions[0][i] * 100) for i in range(NUM_CLASSES)}
    
    return {
        'predicted_class': predicted_class,
        'confidence': confidence,
        'probabilities': probabilities,
        'image': img
    }

def visualize_prediction(image_path, model):
    """
    Visualise la prédiction avec l'image et les probabilités
    """
    result = predict_alzheimer_stage(image_path, model)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Afficher l'image
    axes[0].imshow(result['image'])
    axes[0].set_title(f"Prédiction: {result['predicted_class']}\nConfiance: {result['confidence']:.2f}%",
                     fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Afficher les probabilités
    classes = list(result['probabilities'].keys())
    probs = list(result['probabilities'].values())
    colors = ['green' if c == result['predicted_class'] else 'gray' for c in classes]
    
    axes[1].barh(classes, probs, color=colors, alpha=0.7)
    axes[1].set_xlabel('Probabilité (%)', fontsize=11)
    axes[1].set_title('Probabilités par classe', fontsize=12, fontweight='bold')
    axes[1].set_xlim(0, 100)
    
    # Ajouter les valeurs sur les barres
    for i, (c, p) in enumerate(zip(classes, probs)):
        axes[1].text(p + 1, i, f'{p:.2f}%', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    return result

print("Fonctions de prédiction définies!")
print("\nUtilisation:")
print("  result = predict_alzheimer_stage('path/to/image.jpg', model)")
print("  visualize_prediction('path/to/image.jpg', model)")

## Étape 17: Test de prédiction sur des exemples

In [ ]:
# Tester la prédiction sur quelques images du test set
print("Test de prédiction sur des exemples du test set:\n")

# Sélectionner une image aléatoire de chaque classe
for class_name in CLASSES:
    class_path = TEST_DIR / class_name
    images = list(class_path.glob('*.jpg')) + list(class_path.glob('*.png'))
    
    if images:
        # Prendre une image aléatoire
        random_image = np.random.choice(images)
        print(f"\n{'='*60}")
        print(f"Vraie classe: {class_name}")
        print(f"Image: {random_image.name}")
        print(f"{'='*60}")
        
        result = visualize_prediction(str(random_image), model)

## Étape 18: Résumé final et statistiques

In [ ]:
# Créer un résumé final
summary = f"""
╔══════════════════════════════════════════════════════════════╗
║          RÉSUMÉ DU PROJET - CLASSIFICATION ALZHEIMER        ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  DATASET                                                     ║
║  ────────                                                    ║
║  • Nombre de classes: {NUM_CLASSES}                                        ║
║  • Classes: {', '.join(CLASSES[:2])}                         ║
║            {', '.join(CLASSES[2:])}                          ║
║  • Images d'entraînement: {train_df['Count'].sum()}                              ║
║  • Images de test: {test_df['Count'].sum()}                                    ║
║                                                              ║
║  MODÈLE                                                      ║
║  ──────                                                      ║
║  • Architecture: CNN personnalisé                            ║
║  • Paramètres totaux: {model.count_params():,}                       ║
║  • Taille d'image: {IMG_SIZE[0]}x{IMG_SIZE[1]}                                    ║
║  • Batch size: {BATCH_SIZE}                                           ║
║                                                              ║
║  ENTRAÎNEMENT                                                ║
║  ──────────────                                              ║
║  • Epochs: {len(history.history['loss'])}                                             ║
║  • Learning rate: {LEARNING_RATE}                                   ║
║  • Optimizer: Adam                                           ║
║  • Data augmentation: Oui                                    ║
║                                                              ║
║  PERFORMANCES                                                ║
║  ──────────────                                              ║
║  • Test Accuracy:  {test_acc*100:.2f}%                                  ║
║  • Test Precision: {test_precision*100:.2f}%                                  ║
║  • Test Recall:    {test_recall*100:.2f}%                                  ║
║  • Test F1-Score:  {test_f1*100:.2f}%                                  ║
║                                                              ║
║  FICHIERS SAUVEGARDÉS                                        ║
║  ─────────────────                                           ║
║  • Modèle complet: alzheimer_model_final.h5                 ║
║  • Meilleur modèle: best_model.h5                           ║
║  • Poids: alzheimer_weights.h5                              ║
║  • Architecture: model_architecture.json                    ║
║  • Logs: training_log.csv, training_history.csv             ║
║  • Graphiques: confusion_matrix.png, training_history.png   ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
"""

print(summary)

# Sauvegarder le résumé
with open(MODELS_DIR / 'training_summary.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print("\n✓ Résumé sauvegardé dans: AlzheimerModel/models/training_summary.txt")

## Étape 19: Charger un modèle sauvegardé (pour utilisation future)

In [ ]:
# Code pour charger un modèle sauvegardé
def load_saved_model(model_path):
    """
    Charge un modèle sauvegardé
    """
    model = load_model(model_path)
    print(f"✓ Modèle chargé depuis: {model_path}")
    return model

# Exemple d'utilisation:
# loaded_model = load_saved_model('AlzheimerModel/models/alzheimer_model_final.h5')
# result = predict_alzheimer_stage('path/to/new/image.jpg', loaded_model)

print("Fonction de chargement définie!")
print("\nPour charger un modèle:")
print("  loaded_model = load_saved_model('AlzheimerModel/models/alzheimer_model_final.h5')")

---
## 🎉 Notebook terminé!

### Prochaines étapes:
1. **Améliorer le modèle**: Essayer Transfer Learning (VGG16, ResNet, etc.)
2. **Optimiser**: Tuning des hyperparamètres
3. **Déployer**: Créer une application web pour les prédictions
4. **Analyser**: Étudier les erreurs de classification

### Utilisation du modèle:
```python
# Charger le modèle
model = load_saved_model('AlzheimerModel/models/alzheimer_model_final.h5')

# Faire une prédiction
result = predict_alzheimer_stage('path/to/mri_image.jpg', model)
print(f"Prédiction: {result['predicted_class']}")
print(f"Confiance: {result['confidence']:.2f}%")

# Ou visualiser
visualize_prediction('path/to/mri_image.jpg', model)
```
---